# Orchestration Agent: Report Generator

This Jupyter notebook demonstrates an orchestration agent utilizing LangGraph to generate a structured report on a specified topic. The agent orchestrates multiple steps, including planning report sections, writing content for each section, and compiling the final report.

The workflow employs a `StateGraph` to manage shared state among nodes (agents). This example illustrates sequential orchestration, with an optional asynchronous parallel version for section writing.

**Key Concepts:**
- Shared state with `TypedDict`
- LLM-powered agents for planning, writing, and compiling
- Graph construction with nodes and edges
- Synchronous and asynchronous execution

Ensure you have an `OPENROUTER_API_KEY` environment variable set for the LLM to work.

## 1. Imports and State Definition

We start by importing necessary libraries from LangGraph, LangChain, and standard Python modules. The `ReportState` TypedDict defines the shared state that flows through the graph:
- `topic`: The main subject of the report.
- `sections`: List of section titles planned for the report.
- `section_drafts`: Dictionary mapping section titles to their generated content.
- `final_report`: The compiled markdown report.

This state ensures all agents can access and update relevant information.

In [ ]:
# Imports span typing hints, LangGraph orchestration, and LLM tooling.
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import asyncio
from rich import print
import os
from langchain_ollama import ChatOllama, OllamaEmbeddings
from dotenv import load_dotenv
load_dotenv()

# TypedDict keeps the mutable LangGraph state structured and type-checked.
class ReportState(TypedDict):
    topic: str
    sections: List[str]  # This will be the list of section titles
    section_drafts: Dict[str, str]  # Maps section titles to their content
    final_report: str


## 2. LLM Initialization

Initialize the language model using `ChatOpenAI` configured for OpenRouter. This LLM will power all agents (planner, writer, compiler). Parameters like `temperature=0.7` control creativity, and `max_tokens=250` limits response length per call.


In [ ]:
# Initialize the shared LLM that will power every agent in the workflow.
# llm = ChatOpenAI(
#     model='qwen/qwen3-30b-a3b-instruct-2507',  # OpenRouter-hosted model with strong reasoning skills.
#     base_url='https://openrouter.ai/api/v1',   # Direct requests through the OpenRouter endpoint.
#     temperature=0.7,                           # Allow moderate creativity for nuanced sections.
#     max_tokens=250,                            # Keep each response concise to control latency.
#     api_key=os.getenv('OPENROUTER_API_KEY')    # Pull the API key from environment variables.
# )
OLLAMA_BASE_URL = os.getenv('OLLAMA_BASE_URL', 'http://localhost:11434')
llm = ChatOllama(model='llama3.2', base_url=OLLAMA_BASE_URL, temperature=0.7, max_tokens=400)
# llm = ChatOllama(model='llama3.2:1b', base_url=OLLAMA_BASE_URL, temperature=0.7, max_tokens=500)

## 3. Planner Agent

The `planner_agent` is the first node in the graph. It takes the report topic and uses the LLM to generate 3-4 logical section titles. The prompt instructs the LLM to output only titles, one per line, without numbering.

This agent updates the state with the `sections` list and initializes an empty `section_drafts` dict.

In [ ]:
def planner_agent(state: ReportState) -> ReportState:
    """Plans the structure of the report by defining sections"""
    # Pull the topic from the shared state dictionary supplied by LangGraph.
    topic = state['topic']

    # Build a targeted prompt so the LLM returns only section headings.
    planning_prompt = f"""
    You are a report planning expert. Given the topic: "{topic}"
    
    Create a logical outline with 3-4 main sections for a comprehensive report.
    Return only the section titles, one per line, without numbering.
    
    Example format:
    Introduction and Background
    Current State Analysis
    Future Implications
    Conclusion and Recommendations
    """

    # Ask the LLM for candidate section titles based on the topic.
    response = llm.invoke([HumanMessage(content=planning_prompt)])
    # Split the multiline LLM response into a clean list of section names.
    sections = [line.strip() for line in response.content.strip().split('\n') if line.strip()]

    # Merge the new sections into state without mutating the original object.
    return {**state, 'sections': sections, 'section_drafts': {}}


## 4. Section Writer Helper

The `write_section` helper function generates content for a single section. It classifies the section type (e.g., introductory vs. strategic) to adjust the prompt's focus and tone. The LLM is prompted to write 2-3 paragraphs without including the title.

In [ ]:
def write_section(section_title: str, topic: str) -> str:
    """Helper function to write a single section"""
    # Simple heuristics tune the focus and tone based on the section name.
    if any(keyword in section_title.lower() for keyword in ['introduction', 'background', 'analysis', 'current']):
        focus = 'research-backed information and analysis'
        tone = 'professional and factual'
    else:
        focus = 'strategic insights and actionable recommendations'
        tone = 'forward-thinking and suggestive'

    # Compose instructions that tell the LLM how to draft the section body.
    writer_prompt = f"""
    You are an expert report writer. Write a detailed section for a report on "{topic}".
    
    Section to write: "{section_title}"
    
    Requirements:
    - Write 2-3 substantial paragraphs.
    - Focus on {focus}.
    - Use a {tone}.
    - Do not include the section title itself in your response, just the content.
    """

    # Generate the section copy and trim whitespace before returning it.
    response = llm.invoke([HumanMessage(content=writer_prompt)])
    return response.content.strip()


## 5. Writer Coordinator

The `writer_coordinator` node orchestrates writing all sections by calling `write_section` for each title in sequence. It updates the state with the `section_drafts` dictionary.


In [ ]:
def writer_coordinator(state: ReportState) -> ReportState:
    """Coordinates the writing of all sections"""
    topic = state['topic']
    sections = state['sections']
    # Create a fresh mapping so we do not mutate the incoming state in place.
    section_drafts = {}

    # Iterate sequentially through the planned sections and draft each one.
    for section_title in sections:
        section_content = write_section(section_title, topic)
        # Store the generated paragraphs under their corresponding title.
        section_drafts[section_title] = section_content

    return {**state, 'section_drafts': section_drafts}


## 6. Compiler Agent

The `compiler_agent` finalizes the report by formatting the sections into a markdown string with the topic as the title and each section as a heading followed by its content.


In [ ]:
def compiler_agent(state: ReportState) -> ReportState:
    """Compiles all drafted sections into a final report"""
    topic = state['topic']
    section_drafts = state['section_drafts']
    sections = state['sections']

    # Start the markdown document with a descriptive heading for the topic.
    final_report_content = f'# Report: {topic}\n\n'

    # Walk through sections in outline order to preserve the planned flow.
    for section_title in sections:
        if section_title in section_drafts:
            # Append each section as markdown headings plus the generated body.
            final_report_content += f'## {section_title}\n\n{section_drafts[section_title]}\n\n'

    return {**state, 'final_report': final_report_content}


## 7. Async Versions for Parallel Writing

To improve efficiency, we define async versions of the writer functions. `write_section_async` wraps the synchronous helper, and `writer_coordinator_async` uses `asyncio.gather` to write all sections in parallel.


In [ ]:
# Alternative: Async version for true parallel execution
async def write_section_async(section_title: str, topic: str) -> str:
    """Async helper function to write a single section"""
    # Reuse the synchronous section writer inside an async-friendly wrapper.
    return write_section(section_title, topic)


async def writer_coordinator_async(state: ReportState) -> ReportState:
    """Async coordinator for parallel section writing"""
    topic = state['topic']
    sections = state['sections']

    # Create tasks for parallel execution of each section draft.
    tasks = [write_section_async(section, topic) for section in sections]

    # Execute all tasks in parallel and collect their generated content.
    section_contents = await asyncio.gather(*tasks)

    # Map the list of generated sections back to their original titles.
    section_drafts = dict(zip(sections, section_contents))

    return {**state, 'section_drafts': section_drafts}


## 8. Graph Construction (Synchronous)

Build the `StateGraph` by adding nodes for each agent and defining edges for the sequential flow: planner → writer_coordinator → compiler → END.

Compile the graph into an app for invocation.


In [ ]:
# --- Graph Construction ---

# Build the orchestration graph specifying each agent node and its edges.
workflow = StateGraph(ReportState)

# Add nodes that correspond to each stage of the reporting pipeline.
workflow.add_node('planner', planner_agent)
workflow.add_node('writer_coordinator', writer_coordinator)
workflow.add_node('compiler', compiler_agent)

# The planner kicks off the workflow by generating the report outline.
workflow.set_entry_point('planner')

# Connect nodes to define a simple linear progression.
workflow.add_edge('planner', 'writer_coordinator')
workflow.add_edge('writer_coordinator', 'compiler')
workflow.add_edge('compiler', END)

# Freeze the definition into an executable LangGraph app.
app = workflow.compile()


In [ ]:
print('Subgraph structure:')
# Visualize the compiled graph when graphviz support is available.
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))
# save to file

app.get_graph().draw_mermaid_png(output_file_path='graph.png')


## 10. Execution Functions

`generate_report` runs the synchronous graph to produce the final report. `generate_report_async` builds and runs an async version of the graph, using the parallel writer coordinator.


In [ ]:
# --- Execution Functions ---


def generate_report(topic: str) -> str:
    """Generate a report using the sequential workflow."""
    # Seed the graph state with empty containers for sections and drafts.
    initial_state = {'topic': topic, 'sections': [], 'section_drafts': {}, 'final_report': ''}
    # debug=True prints each node transition for instructional purposes.
    result = app.invoke(initial_state, debug=True)
    return result['final_report']


async def generate_report_async(topic: str) -> str:
    """Generate a report with parallel section writing."""
    # Create async workflow that swaps in the concurrent writer coordinator.
    async_workflow = StateGraph(ReportState)
    async_workflow.add_node('planner', planner_agent)
    async_workflow.add_node('writer_coordinator', writer_coordinator_async)
    async_workflow.add_node('compiler', compiler_agent)

    async_workflow.set_entry_point('planner')
    async_workflow.add_edge('planner', 'writer_coordinator')
    async_workflow.add_edge('writer_coordinator', 'compiler')
    async_workflow.add_edge('compiler', END)

    # Compile the async workflow so it can be awaited.
    async_app = async_workflow.compile()

    initial_state = {'topic': topic, 'sections': [], 'section_drafts': {}, 'final_report': ''}
    # Kick off the asynchronous execution pathway.
    result = await async_app.ainvoke(initial_state)
    return result['final_report']


## 11. Running the Example

The main execution generates a sample report on 'The Impact of Artificial Intelligence on Healthcare' using the synchronous function. For async, uncomment and run with `asyncio.run`.


In [ ]:
# Choose a sample topic to demonstrate the orchestration pipeline.
topic = 'The Impact of Artificial Intelligence on Healthcare'

# Generate the report synchronously so we can inspect the output immediately.
print(f'Generating report on: {topic}...')
# report = generate_report(topic)
# print(report)

# For async version:
# In a Jupyter notebook, which runs an event loop, use await instead of asyncio.run()
report_async = await generate_report_async(topic)  # Run the parallel variant.
print(report_async)

# Run the example (uncomment to execute)
# topic = 'The Impact of Artificial Intelligence on Healthcare'
# report = generate_report(topic)
# print(report)
